In [142]:
import numpy as np

from qiskit.compiler import transpile, assemble
from qiskit import Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, HFMethodType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute


from qiskit.chemistry.components.initial_states import HartreeFock

# Define Molecule

In [143]:
# PySCF calc
driver = PySCFDriver(
                     atom='H .0 .0 .0; H .0 .0 0.74',
#                      atom='Li .0 .0 .0; H .0 .0 1.44',
                     unit=UnitsType.ANGSTROM,
                     charge=0,
                     spin=0,
                     basis='sto3g',
                     hf_method= HFMethodType.RHF,
                     conv_tol=1e-9,
                     max_cycle= 50)
molecule = driver.run()

In [72]:
# from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
# #https://qiskit.org/documentation/_modules/qiskit/chemistry/core/hamiltonian.html


# Chem_Ham = Hamiltonian(TransformationType.FULL,#TransformationType
#             QubitMappingType.BRAVYI_KITAEV,#QubitMappingType QubitMappingType.PARITY, QubitMappingType.JORDAN_WIGNER, QubitMappingType.BRAVYI_KITAEV
#             True,  #two_qubit_reduction
#             False, #freeze_core
#             None,  #orbital_reduction
#             'auto')  # z2symmetry_reduction

# Chem_Ham.run(molecule)

# # log_lines, results = Chem_Ham.process_algorithm_result()

In [144]:
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -1.8318636464775067
# of electrons: 2
# of spin orbitals: 4


# Get Qubit Hamiltonian

In [119]:
ferOp = FermionicOperator(h1=h1, h2=h2)

In [145]:
# map_type = 'bravyi_kitaev'
map_type='jordan_wigner'
# qubit_mapping='parity'

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)

print(len(qubitOp.to_dict()['paulis']))
print(qubitOp.num_qubits)

exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
ret = exact_eigensolver.run()
print('The total ground state energy is: {:.12f}'.format(ret.eigenvalue.real + nuclear_repulsion_energy))

15
4
The total ground state energy is: -1.137283834489


In [121]:
# print(qubitOp.print_details())

In [146]:
from qiskit.quantum_info import Operator
from scipy.sparse.linalg import eigs

H_mat_sparse=qubitOp.to_opflow().to_spmatrix()


eig_values, eig_vectors = eigs(H_mat_sparse)
FCI_Energy = min(eig_values)
index = np.where(eig_values==FCI_Energy)[0][0]
ground_state = eig_vectors[:, index]

# H_mat = Operator(qubitOp.to_opflow().to_matrix())
# # print(H_mat._data)

# from scipy.linalg import eig
# from openfermion import get_sparse_operator

# eig_values, eig_vectors = eig(H_mat._data)
# FCI_Energy = min(eig_values)
# index = np.where(eig_values==FCI_Energy)[0][0]
ground_state = eig_vectors[:, index]

print('fci ground E =', FCI_Energy+ nuclear_repulsion_energy)
print('ground state = ', ground_state)

fci ground E = (-1.1372838344885001+3.3140182052923533e-17j)
ground state =  [-2.18581528e-16-2.30138819e-16j -1.54890834e-16-6.83169323e-17j
  5.97642671e-17+8.30731978e-17j -1.24274699e-16-1.03154909e-16j
  2.73782248e-16-1.53785432e-16j  9.41870097e-01-3.16566888e-01j
 -4.32394593e-17+3.13075547e-17j -1.91799173e-16-7.32994847e-17j
  2.01521344e-18+1.34946376e-16j  4.46065752e-17-2.29636609e-17j
 -1.06679483e-01+3.58554666e-02j -8.41178512e-17-3.86737554e-17j
 -8.60671455e-17+1.81991656e-16j  9.11396526e-17+1.08491034e-16j
 -6.35562357e-18+2.70764477e-17j  3.45758668e-16+5.67083911e-17j]


# TAPERING!

In [103]:
from qiskit.aqua.operators import Z2Symmetries
print(Z2Symmetries.find_Z2_symmetries(qubitOp))

# Chemistry specific method: It can be used to taper two qubits in
# --> parity
# --> binary-tree == Bravyi-Kitaev (https://qiskit.org/documentation/apidoc/qiskit_chemistry.html)
# mapped Fermionic Hamiltonians  

# NOTE when the spin orbitals are ordered in two spin sectors, (block spin order) 
#      according to the number of particles in the system

if (map_type == 'bravyi_kitaev') or (map_type == 'parity'):
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)

Z2 symmetries:
Symmetries:
ZIIZ
ZIZI
ZZII
Single-Qubit Pauli X:
IIIX
IIXI
IXII
Cliffords:
ZIIZ	(0.7071067811865475+0j)
IIIX	(0.7071067811865475+0j)

ZIZI	(0.7071067811865475+0j)
IIXI	(0.7071067811865475+0j)

ZZII	(0.7071067811865475+0j)
IXII	(0.7071067811865475+0j)

Qubit index:
[0, 1, 2]
Tapering values:
  - Possible values: [1, 1, 1], [1, 1, -1], [1, -1, 1], [1, -1, -1], [-1, 1, 1], [-1, 1, -1], [-1, -1, 1], [-1, -1, -1]


In [147]:
print(len(qubitOp.to_dict()['paulis']))
print(qubitOp.num_qubits)
print(qubitOp.print_details())

15
4
IIII	(-0.8121706072487107+0j)
IIIZ	(0.17141282644776898+0j)
IIZI	(-0.22343153690813541+0j)
IZII	(0.171412826447769+0j)
ZIII	(-0.2234315369081354+0j)
IIZZ	(0.12062523483390412+0j)
IZIZ	(0.16868898170361207+0j)
XXYY	(0.04530261550379926+0j)
YYYY	(0.04530261550379926+0j)
XXXX	(0.04530261550379926+0j)
YYXX	(0.04530261550379926+0j)
ZIIZ	(0.16592785033770338+0j)
IZZI	(0.16592785033770338+0j)
ZIZI	(0.17441287612261575+0j)
ZZII	(0.12062523483390412+0j)



#### re-check fci if two qubit reduction performed

In [148]:
if (map_type == 'bravyi_kitaev') or (map_type == 'parity'):
    H_mat_sparse=qubitOp.to_opflow().to_spmatrix()

    eig_values, eig_vectors = eigs(H_mat_sparse)
    FCI_Energy = min(eig_values)
    index = np.where(eig_values==FCI_Energy)[0][0]
    ground_state = eig_vectors[:, index]


    print('fci ground E =', FCI_Energy+ nuclear_repulsion_energy)
    print('ground state = ', ground_state)
    
    
    # Using exact eigensolver to get the smallest eigenvalue
    exact_eigensolver_new = NumPyMinimumEigensolver(qubitOp)
    ret_new = exact_eigensolver_new.run()
    print('The computed energy is: {:.12f}'.format(ret_new.eigenvalue.real))
    print('The total ground state energy is: {:.12f}'.format(ret_new.eigenvalue.real + nuclear_repulsion_energy))

### Get Hartree-Fock initial state

In [149]:
num_particles = molecule.num_alpha + molecule.num_beta
# Args:
#     num_orbitals: number of spin orbitals, has a min. value of 1.
#     num_particles: number of particles, if it is a list, the first number
#                     is alpha and the second number if beta.
#     qubit_mapping: mapping type for qubit operator
#     two_qubit_reduction: flag indicating whether or not two qubit is reduced
#     sq_list: position of the single-qubit operators that
#             anticommute with the cliffords
HF_obj = HartreeFock(qubitOp.num_qubits, #molecule.num_orbitals,
                 num_particles,
                 qubit_mapping=map_type,
                 two_qubit_reduction=False) # already done!

q_reg = QuantumRegister(qubitOp.num_qubits)

print('HF state =', np.array(HF_obj.bitstr, dtype=int))
HF_circuit = HF_obj.construct_circuit(mode='circuit', register=q_reg)
HF_circuit.draw()

HF state = [0 1 0 1]


┌─────────────┐
q8_0: ┤ U3(pi,0,pi) ├
      └─────────────┘
q8_1: ───────────────
      ┌─────────────┐
q8_2: ┤ U3(pi,0,pi) ├
      └─────────────┘
q8_3: ───────────────

In [154]:
qubitOp.num_qubits

4

In [158]:
from qiskit.chemistry.components.variational_forms.uccsd import UCCSD
num_particles = molecule.num_alpha + molecule.num_beta

UCCSD_ansatz= UCCSD(
                 qubitOp.num_qubits, # number orbitals int,
                 num_particles,
                 reps=1, #number of repetitions of basic module
                 active_occupied=None,
                 active_unoccupied=None,
                 initial_state=None,
                 qubit_mapping=map_type,
                 two_qubit_reduction=True,
                 num_time_slices=1,
                 shallow_circuit_concat=True,
                 z2_symmetries=None,
                 method_singles='both',
                 method_doubles='ucc',
                 excitation_type='sd',
                 same_spin_doubles=True,
                 skip_commute_test=False)

In [161]:
UCCSD_ansatz.num_parameters

3

In [169]:
singles, doubles = UCCSD_ansatz.compute_excitation_lists(
                        num_particles,
                        qubitOp.num_qubits,
                        active_occ_list=None,
                        active_unocc_list=None, 
                        same_spin_doubles=True,
                        method_singles='both', 
                        method_doubles='ucc',
                        excitation_type='sd')
print(singles)
print(doubles)

[[0, 1], [2, 3]]
[[0, 1, 2, 3]]


In [170]:
test = UCCSD_ansatz.construct_circuit([np.pi, np.pi/2, np.pi/3])
test.decompose().draw()

┌──────────┐  ┌────────┐ ┌──────────┐┌─────────────────────┐  ┌────────┐ »
q_0: ┤ U2(0,pi) ├──┤ U1(pi) ├─┤ U2(0,pi) ├┤ U3(pi/2,-pi/2,pi/2) ├──┤ U1(pi) ├─»
     ├──────────┤┌─┴────────┴┐├──────────┤├─────────────────────┤┌─┴────────┴┐»
q_1: ┤ U2(0,pi) ├┤ U1(-pi/2) ├┤ U2(0,pi) ├┤ U3(pi/2,-pi/2,pi/2) ├┤ U1(-pi/2) ├»
     └──────────┘└───────────┘└──────────┘└─────────────────────┘└───────────┘»
«     ┌──────────────────────┐┌──────────┐                      ┌──────────┐»
«q_0: ┤ U3(-pi/2,-pi/2,pi/2) ├┤ U2(0,pi) ├──■────────────────■──┤ U2(0,pi) ├»
«     ├──────────────────────┤├──────────┤┌─┴─┐┌──────────┐┌─┴─┐├──────────┤»
«q_1: ┤ U3(-pi/2,-pi/2,pi/2) ├┤ U2(0,pi) ├┤ X ├┤ U1(pi/6) ├┤ X ├┤ U2(0,pi) ├»
«     └──────────────────────┘└──────────┘└───┘└──────────┘└───┘└──────────┘»
«     ┌─────────────────────┐                       ┌──────────────────────┐
«q_0: ┤ U3(pi/2,-pi/2,pi/2) ├──■─────────────────■──┤ U3(-pi/2,-pi/2,pi/2) ├
«     ├─────────────────────┤┌─┴─┐┌───────────┐┌─┴─┐├──────────────────────┤
«q_1: ┤ U3(pi/2,-pi/2,pi/2) ├┤ X ├┤ U1(-pi/6) ├┤ X ├┤ U3(-pi/2,-pi/2,pi/2) ├
«     └─────────────────────┘└───┘└───────────┘└───┘└──────────────────────┘

In [ ]:
from openfermion.ops import QubitOperator

QubitHamiltonian = QubitOperator()
for P_term_dict in qubitOp.to_dict()['paulis']:
    
    # NOTE CHANGE ORDER for QubitOperator!
    Pauli = ' '.join(['{}{}'.format(P_str, index) for index,  P_str in enumerate(P_term_dict['label'][::-1]) if P_str != 'I'])
    QubitHamiltonian+= QubitOperator(Pauli, P_term_dict['coeff']['real'])

QubitHamiltonian

## check FCI

In [ ]:
from scipy.linalg import eig
from openfermion import get_sparse_operator

H_mat=get_sparse_operator(QubitHamiltonian, n_qubits=qubitOp.num_qubits).todense()
eig_values, eig_vectors = eig(H_mat)
FCI_Energy = min(eig_values)
index = np.where(eig_values==FCI_Energy)[0][0]
ground_state = eig_vectors[:, index]

print('fci ground E =', FCI_Energy+ nuclear_repulsion_energy)
print('ground state = ', ground_state)

In [ ]:
## checking via lin alg
E_elec=ground_state.reshape(ground_state.shape[0],1).conj().T.dot(H_mat.dot(ground_state.reshape(ground_state.shape[0],1)))
E_elec+nuclear_repulsion_energy

In [ ]:
# from scipy.sparse.linalg import eigs
# from openfermion import get_sparse_operator

# eig_values, eig_vectors = eigs(get_sparse_operator(QubitHamiltonian))#, n_qubits=qubitOp.num_qubits))
# FCI_Energy = min(eig_values)
# index = np.where(eig_values==FCI_Energy)[0][0]
# ground_state = eig_vectors[:, index]
# print('fci ground E =', FCI_Energy+ nuclear_repulsion_energy)
# print('ground state = ', ground_state)

# Get anti-commuting sets

In [ ]:
from quchem.Graph import *

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'

anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)




In [ ]:
from qiskit.quantum_info import Pauli
# PP = Pauli(z=[True, True, True, False, False, False, False, False, False, False], x=[False, True, True, False, False, False, False, False, False, False])
# print(PP)

In [ ]:
from qiskit.aqua.operators.primitive_ops import PauliOp
# PPP = PauliOp(PP, coeff=0.5)
# print(PPP)

In [ ]:
# ## convert openfermion.QubitOperator to qiskit PauliOp
# NN=5
# Q_op = QubitOperator('X0 Y1 Z4')
# P_strs, coeff = zip(list(*Q_op.terms.items()))

# if P_strs[0]:
#     q_Nums, p_strs = zip(*P_strs[0])

# #         P_list = ['{}{}'.format(p_strs[q_Nums.index(qNo)], qNo) if qNo in q_Nums else '{}{}'.format('I', qNo) for qNo in range(N_qubits) ]
#     P_list = ['{}'.format(p_strs[q_Nums.index(qNo)]) if qNo in q_Nums else '{}'.format('I') for qNo in range(NN) ]
# print(P_list)

# z = np.zeros(len(P_list), dtype=np.bool)
# x = np.zeros(len(P_list), dtype=np.bool)
# for i, char in enumerate(P_list):
#     if char == 'X':
#         x[i] = True
#     elif char == 'Z':
#         z[i] = True
#     elif char == 'Y':
#         z[i] = True
#         x[i] = True
#     elif char != 'I':
#         raise QiskitError("Pauli string must be only consisted of 'I', 'X', "
#                           "'Y' or 'Z' but you have {}.".format(char))
# Pauli_IBM = Pauli(z=z,x=x)
# Pauli_Op_IBM = PauliOp(Pauli_IBM, coeff=coeff)
# print(str(Pauli_Op_IBM.primitive))

In [ ]:
## convert openfermion.QubitOperator to qiskit PauliOp
N_qubits = qubitOp.num_qubits

IBM_anti_commuting_sets={}
for key in anti_commuting_sets:
    OP_list=[]
    for Q_op in anti_commuting_sets[key]:
        P_strs, coeff = zip(list(*Q_op.terms.items()))
        
        if P_strs[0]:
            q_Nums, p_strs = zip(*P_strs[0])

    #         P_list = ['{}{}'.format(p_strs[q_Nums.index(qNo)], qNo) if qNo in q_Nums else '{}{}'.format('I', qNo) for qNo in range(N_qubits) ]
            P_list = ['{}'.format(p_strs[q_Nums.index(qNo)]) if qNo in q_Nums else '{}'.format('I') for qNo in range(N_qubits) ]

            z = np.zeros(len(P_list), dtype=np.bool)
            x = np.zeros(len(P_list), dtype=np.bool)
            for i, char in enumerate(P_list):
#                 if char == 'X':
#                     x[-i - 1] = True
#                 elif char == 'Z':
#                     z[-i - 1] = True
#                 elif char == 'Y':
#                     z[-i - 1] = True
#                     x[-i - 1] = True
                if char == 'X':
                    x[i] = True
                elif char == 'Z':
                    z[i] = True
                elif char == 'Y':
                    z[i] = True
                    x[i] = True
                elif char != 'I':
                    raise QiskitError("Pauli string must be only consisted of 'I', 'X', "
                                      "'Y' or 'Z' but you have {}.".format(char))
            Pauli_IBM = Pauli(z=z,x=x)
            Pauli_Op_IBM = PauliOp(Pauli_IBM, coeff=coeff)
#             print(Pauli_Op_IBM)
            OP_list.append(Pauli_Op_IBM)
        else:
            z = np.zeros(N_qubits, dtype=np.bool)
            x = np.zeros(N_qubits, dtype=np.bool)
            Pauli_IBM = Pauli(z=z,x=x)
            Pauli_Op_IBM = PauliOp(Pauli_IBM, coeff=coeff)
#             print(Pauli_Op_IBM)
            OP_list.append(Pauli_Op_IBM)
        
    IBM_anti_commuting_sets[key]=OP_list

In [ ]:
print(anti_commuting_sets[1][0])
str(IBM_anti_commuting_sets[1][0].primitive)

In [ ]:
print(anti_commuting_sets[7][3])
str(IBM_anti_commuting_sets[7][3].primitive)

In [ ]:
#### compare dictionaries 

In [ ]:
# anti_commuting_sets

In [ ]:
# {key:[(str(op.primitive), op.coeff) for op in IBM_anti_commuting_sets[key]]  for key in IBM_anti_commuting_sets}

In [ ]:
#### compare dictionaries

In [ ]:
set_key=3
index=0
print(IBM_anti_commuting_sets[set_key][index])
print(anti_commuting_sets[set_key][index])

xx=IBM_anti_commuting_sets[set_key][index].to_circuit()
print(xx)

In [ ]:

N_qubits = qubitOp.num_qubits
# arb state initialization
# https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits/3_summary_of_quantum_operations.ipynb

q = QuantumRegister(N_qubits)

qc = QuantumCircuit(q)

qc.initialize(ground_state.tolist(), [q[i] for i in range(N_qubits)])
qc.draw()


In [ ]:
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
qc_state = job.result().get_statevector(qc)
qc_state

In [ ]:
np.allclose(ground_state, qc_state)

In [ ]:
qc.measure_all()
backend=Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=2000)
result = job.result()
out = result.get_counts(qc)
out

In [ ]:
def change_basis_and_measure_IBM(PauliWord, q_register, q_circuit):

    # change basis
    
    # note change of order in enumerate
    for qNo, Pstr in enumerate(str(PauliWord.primitive)[::-1]):
        if Pstr == 'X':
            q_circuit.h(q_register[qNo])
        #             q_circuit.measure(q_register[qNo], c_register[index])

        elif Pstr == 'Y':
            q_circuit.rx((+np.pi / 2), q_register[qNo])
        #             q_circuit.measure(q_register[qNo], c_register[index])

        elif (Pstr == 'Z') or (Pstr == 'I'):
            continue
        #             q_circuit.measure(q_register[qNo], c_register[index])

        else:
            raise ValueError('Not a PauliWord')

    q_circuit.measure_all()

    return q_circuit

In [ ]:
q = QuantumRegister(N_qubits)
qc = QuantumCircuit(q)
qc.initialize(ground_state.tolist(), [q[i] for i in range(N_qubits)])

z = np.array([1,0,0,0], dtype=np.bool)
x = np.array([0,0,0,0], dtype=np.bool)
Pauli_test = Pauli(z=z,x=x)
Paulitest_Op = PauliOp(Pauli_test, coeff=coeff)
print(str(Paulitest_Op.primitive))
     
cir= change_basis_and_measure_IBM(Paulitest_Op, q, qc)
print(cir.draw())

backend=Aer.get_backend('qasm_simulator')
job = execute(cir, backend, shots=2000)
result = job.result()
out = result.get_counts(cir)
print(out)

calc_exp_pauliword(out, str(Paulitest_Op.primitive))

In [ ]:
from qiskit.aqua.operators.primitive_ops import PauliOp
# PPP = PauliOp(PP, coeff=0.5)
# print(PPP)
from qiskit.quantum_info import Pauli
z=np.array([1,0,0], dtype=bool)
x=np.array([1,1,0], dtype=bool)
P = Pauli(z=z, x=x)
PW = PauliOp(P, coeff=0.5)
print(str(PW.primitive))

q = QuantumRegister(len(PW.primitive))
qc = QuantumCircuit(q)

cir= change_basis_and_measure_IBM(PW, q, qc)
cir.draw()

In [ ]:
N_qubits = qubitOp.num_qubits
# arb state initialization
# https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits/3_summary_of_quantum_operations.ipynb


qc_list=[]
Pword_list=[]

arb_input_circuit=None
for set_key in IBM_anti_commuting_sets:
    for op in IBM_anti_commuting_sets[set_key]:
        
        IBM_q_circuit = op.to_circuit()
        
        if arb_input_circuit is None:
            q_reg = QuantumRegister(N_qubits)
            arb_input_circuit = QuantumCircuit(q_reg)

            arb_input_circuit.initialize(ground_state.tolist(), [q_reg[i] for i in range(N_qubits)])

        
        full_ibm_circuit=change_basis_and_measure_IBM(op, q_reg, arb_input_circuit.copy())
        qc_list.append(full_ibm_circuit)
        Pword_list.append((str(op.primitive), op.coeff[0]))
        

In [ ]:
qc_list[-1].draw()

In [ ]:
circuit = qc_list[-1].copy()
circuit.remove_final_measurements()
backend = Aer.get_backend('statevector_simulator')
job = execute(circuit, backend)
qc_state = job.result().get_statevector(circuit)
qc_state

In [ ]:
print(qc_list[-1].draw())

backend=Aer.get_backend('qasm_simulator')
job = execute(qc_list[-1], backend, shots=2000)
result = job.result()
out = result.get_counts(qc_list[-1])
out

In [ ]:
circ = QuantumCircuit(3)
circ.x(0)

circ.measure_all()

print(circ.draw())

backend=Aer.get_backend('qasm_simulator')
job = execute(circ, backend, shots=2000)
result = job.result()
out = result.get_counts(circ)
out

In [ ]:

n_shots=2000
backend_simulator=Aer.get_backend('qasm_simulator')

transpiled_circs_sim = transpile(qc_list, backend=backend_simulator, optimization_level=0)
qobjs_sim = assemble(transpiled_circs_sim, backend=backend_simulator, shots=n_shots)

sim_job = backend_simulator.run(qobjs_sim)

In [ ]:
raw_results = [sim_job.result().get_counts(transpiled_circs_sim[circ_index]) for circ_index in
                                    range(len(transpiled_circs_sim))]

print(Pword_list)
raw_results

In [ ]:
raw_res=[]
backend=Aer.get_backend('qasm_simulator')
n_shots=2000

for circuit in transpiled_circs_sim:
    job = execute(circuit, backend, shots=n_shots)
    result = job.result()
    raw_res.append(result.get_counts(circuit))
    
raw_res

In [ ]:
print(qc_list[-1].draw())

backend=Aer.get_backend('qasm_simulator')
job = execute(qc_list[-1], backend, shots=2000)
result = job.result()
out = result.get_counts(qc_list[-1])
out

In [ ]:
def calc_exp_pauliword(count_dict, PauliWord_Str):
    # takes correct part of bitstring when all lines measured
    
    if np.alltrue(np.array(list(PauliWord_Str), dtype=str)=='I'):
        return 1
    else:
        Non_I_indices = np.where(np.array(list(PauliWord_Str), dtype=str) != 'I')
        n_zeros = 0
        n_ones = 0

        for bitstring in count_dict:
            # note reversed order!
            measure_term = np.take([int(bit) for bit in bitstring], Non_I_indices)[0]

            parity_m_term = sum(measure_term) % 2

            if parity_m_term == 0:
                n_zeros += count_dict[bitstring]
            elif parity_m_term == 1:
                n_ones += count_dict[bitstring]
            else:
                raise ValueError('state {} not allowed'.format(measure_term))

        expectation_value = (n_zeros - n_ones) / (n_zeros + n_ones)

        return expectation_value

def Calc_Energy(raw_results, PWord_list):
    E_list=[]
    for index, raw_result_dic in enumerate(raw_results):
        P_word=PWord_list[index][0]
        coeff=PWord_list[index][1]
        
        exp_val = calc_exp_pauliword(raw_result_dic, P_word)
        print(exp_val, coeff)
        E_list.append(exp_val*coeff)
        
    return sum(E_list)
        
    

In [ ]:
Pword_list

In [ ]:
raw_results

In [ ]:
raw_res

In [ ]:
Calc_Energy(raw_results, Pword_list)+ nuclear_repulsion_energy

In [ ]:
Calc_Energy(raw_res, Pword_list)+ nuclear_repulsion_energy

In [ ]:
FCI_Energy + nuclear_repulsion_energy

In [ ]:
import itertools
list_op=qubitOp.to_opflow().oplist

indices = range(len(list_op))
[(i, j) for i, j in itertools.combinations(indices, 2) if not list_op[i].commutes(list_op[j])]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm


def IBM_Build_Graph_Nodes(List_of_nodes, Graph, plot_graph=False):
    """

    Function builds nodes of graph with attributes

    """
    labels={}
    node_list=[]
    for node in List_of_nodes:
        Graph.add_node(node)

        if plot_graph is True:
            node_list.append(node)

            PauliWord = str(node.primitive)
            labels[node] = PauliWord


    if plot_graph is True:
        plt.figure()

        pos = nx.circular_layout(Graph)

        nx.draw_networkx_nodes(Graph, pos,
                               nodelist=node_list,
                               node_color='r',
                               node_size=500,
                               alpha=0.8)

        nx.draw_networkx_labels(Graph, pos, labels)  # , font_size=8)
        plt.show()
    return Graph


In [ ]:
G=nx.Graph()
IBM_Build_Graph_Nodes(list_op, G, plot_graph=True)

In [ ]:
def Openfermion_Build_Graph_Edges_COMMUTING_QWC_AntiCommuting(Graph, List_of_nodes, anti_comm_QWC, plot_graph = False):
    """

    Function builds graph edges for commuting / anticommuting / QWC PauliWords

    Args:
        PauliWord_string_nodes_list (list): list of PauliWords (str)
        Graph: networkX graph with nodes already defined
        anti_comm_QWC (str): flags to find either:
                                           qubit wise commuting (QWC) terms  -> flag = 'QWC',
                                                             commuting terms -> flag = 'C',
                                                        anti-commuting terms -> flag = 'AC'
        plot_graph (optional, bool): whether to plot graph

    Returns:
        Graph: Graph with nodes connected if they commute / QWC / anti-commute

    """
    node_list=[]
    labels={}

    for index, selected_PauliWord in enumerate(tqdm(List_of_nodes, ascii=True, desc='Building Graph Edges')):

        for j in range(index + 1, len(List_of_nodes)):
            comparison_PauliWord = List_of_nodes[j]

            if OpenFermion_Commutativity(selected_PauliWord, comparison_PauliWord, anti_comm_QWC) is True:
                Graph.add_edge(selected_PauliWord, comparison_PauliWord)
            else:
                continue

        if plot_graph is True:
            node_list.append(selected_PauliWord)
            PauliStrs, _ = selected_PauliWord
            PauliStr_list = [''.join(map(str, [element for element in tupl[::-1]])) for tupl in PauliStrs]
            PauliWord = ' '.join(PauliStr_list)
            labels[selected_PauliWord] = PauliWord

    if plot_graph is True:
        plt.figure()

        pos = nx.circular_layout(Graph)

        nx.draw_networkx_nodes(Graph, pos,
                               nodelist=node_list,
                               node_color='r',
                               node_size=500,
                               alpha=0.8)

        nx.draw_networkx_labels(Graph, pos, labels)  # , font_size=8)
        nx.draw_networkx_edges(Graph, pos, width=1.0, alpha=0.5)
        plt.show()

    return Graph

In [ ]:
i=3
j=-5

print(str(list_op[i].primitive))
op1 = list_op[i]

print(str(list_op[j].primitive))
op2 = list_op[j]

In [ ]:
def IBM_Commutativity(Op1, Op2, Comm_flag):

    
    if Comm_flag=='C':
        # https://qiskit.org/documentation/_modules/qiskit/aqua/operators/primitive_ops/pauli_op.html#PauliOp
        return Op1.commutes(Op2)
    else:

        
        if Comm_flag=='QWC':
            Z_op1=Op1.primitive.z
            X_op1=Op1.primitive.x 

            Z_op2=Op2.primitive.z
            X_op2=Op2.primitive.x 
            
            Z_terms = np.array(Z_op1, dtype=int) + np.array(Z_op2, dtype=int)
            X_terms = np.array(X_op1, dtype=int)+ np.array(X_op2, dtype=int)
            
            print(Z_terms)
            print(X_terms)
            
            if (sum(Z_terms)<=len(Z_op2)) and (sum(X_terms)<=len(X_op2)) and ((Z_terms<=1).all()) and ((X_terms<=1).all()):
                return True
            elif Op1==Op2:
                return True
            else:
                return False
        if Op1.commutes(Op2) is False:
            return True
        else:
            return False

        
i=5
j=6
print(str(list_op[i].primitive))
op1 = list_op[i]
print(str(list_op[j].primitive))
op2 = list_op[j]

IBM_Commutativity(op1, op2, 'QWC')

In [ ]:


def Openfermion_Build_Graph_Nodes(List_of_nodes, Graph, plot_graph=False):
    """

    Function builds nodes of graph with attributes

    Args:
        List_of_nodes (list): A list of Pauliwords, where each entry is a tuple of (PauliWord, constant)
        Graph ():
        node_attributes_dict
        plot_graph (optional, bool): whether to plot graph

    Returns:



    .. code-block:: python

       from quchem.Graph import *

       node_attributes_dict =  {'Cofactor': {'I0 I1 I2 I3': (-0.09706626861762624+0j),
                                                         'Z0 I1 I2 I3': (0.17141282639402405+0j),
                                                         'I0 Z1 I2 I3': (0.171412826394024+0j),
                                                         'I0 I1 Z2 I3': (-0.2234315367466397+0j),
                                                         'I0 I1 I2 Z3': (-0.2234315367466397+0j),
                                                         'Z0 Z1 I2 I3': (0.1686889816869329+0j),
                                                         'Y0 X1 X2 Y3': (0.04530261550868928+0j),
                                                         'Y0 Y1 X2 X3': (-0.04530261550868928+0j),
                                                         'X0 X1 Y2 Y3': (-0.04530261550868928+0j),
                                                         'X0 Y1 Y2 X3': (0.04530261550868928+0j),
                                                         'Z0 I1 Z2 I3': (0.12062523481381837+0j),
                                                         'Z0 I1 I2 Z3': (0.16592785032250768+0j),
                                                         'I0 Z1 Z2 I3': (0.16592785032250768+0j),
                                                         'I0 Z1 I2 Z3': (0.12062523481381837+0j),
                                                         'I0 I1 Z2 Z3': (0.174412876106516+0j)
                                                         }
                                                }

        DO SOMETHING
        >> blah

    """
    labels={}
    node_list=[]
    for node in List_of_nodes:
        Graph.add_node(node)

        if plot_graph is True:
            node_list.append(node)

            PauliStrs, _ = node
            PauliStr_list = [''.join(map(str,[element for element in tupl[::-1]])) for tupl in PauliStrs]
            PauliWord= ' '.join(PauliStr_list)
            labels[node] = PauliWord


    if plot_graph is True:
        plt.figure()

        pos = nx.circular_layout(Graph)

        nx.draw_networkx_nodes(Graph, pos,
                               nodelist=node_list,
                               node_color='r',
                               node_size=500,
                               alpha=0.8)

        nx.draw_networkx_labels(Graph, pos, labels)  # , font_size=8)
        plt.show()
    return Graph

def Openfermion_Build_Graph_Edges_COMMUTING_QWC_AntiCommuting(Graph, List_of_nodes, anti_comm_QWC, plot_graph = False):
    """

    Function builds graph edges for commuting / anticommuting / QWC PauliWords

    Args:
        PauliWord_string_nodes_list (list): list of PauliWords (str)
        Graph: networkX graph with nodes already defined
        anti_comm_QWC (str): flags to find either:
                                           qubit wise commuting (QWC) terms  -> flag = 'QWC',
                                                             commuting terms -> flag = 'C',
                                                        anti-commuting terms -> flag = 'AC'
        plot_graph (optional, bool): whether to plot graph

    Returns:
        Graph: Graph with nodes connected if they commute / QWC / anti-commute

    """
    node_list=[]
    labels={}

    for index, selected_PauliWord in enumerate(tqdm(List_of_nodes, ascii=True, desc='Building Graph Edges')):

        for j in range(index + 1, len(List_of_nodes)):
            comparison_PauliWord = List_of_nodes[j]

            if OpenFermion_Commutativity(selected_PauliWord, comparison_PauliWord, anti_comm_QWC) is True:
                Graph.add_edge(selected_PauliWord, comparison_PauliWord)
            else:
                continue

        if plot_graph is True:
            node_list.append(selected_PauliWord)
            PauliStrs, _ = selected_PauliWord
            PauliStr_list = [''.join(map(str, [element for element in tupl[::-1]])) for tupl in PauliStrs]
            PauliWord = ' '.join(PauliStr_list)
            labels[selected_PauliWord] = PauliWord

    if plot_graph is True:
        plt.figure()

        pos = nx.circular_layout(Graph)

        nx.draw_networkx_nodes(Graph, pos,
                               nodelist=node_list,
                               node_color='r',
                               node_size=500,
                               alpha=0.8)

        nx.draw_networkx_labels(Graph, pos, labels)  # , font_size=8)
        nx.draw_networkx_edges(Graph, pos, width=1.0, alpha=0.5)
        plt.show()

    return Graph

def Openfermion_Get_Complemenary_Graph(Graph, plot_graph=False):

    Complement_Graph = nx.complement(Graph)

    node_list=[]
    labels={}
    if plot_graph is True:
        plt.figure()
        for node in Complement_Graph.nodes:
            node_list.append(node)
            PauliStrs, _ = node
            PauliStr_list = [''.join(map(str, [element for element in tupl[::-1]])) for tupl in PauliStrs]
            PauliWord = ' '.join(PauliStr_list)
            labels[node] = PauliWord

        pos = nx.circular_layout(Complement_Graph)

        nx.draw_networkx_nodes(Complement_Graph, pos,
                               nodelist=node_list,
                               node_color='r',
                               node_size=500,
                               alpha=0.8)

        nx.draw_networkx_labels(Complement_Graph, pos, labels)  # , font_size=8)
        nx.draw_networkx_edges(Complement_Graph, pos, width=1.0, alpha=0.5)
        plt.show()
    return Complement_Graph

def Openfermion_Get_clique_cover(Graph, strategy='largest_first', plot_graph=False):
    """
    https: // en.wikipedia.org / wiki / Clique_cover

    Function gets clique cover of a graph. Does this via a graph colouring approach - therefore
    strategy is important here!

    Args:
        Graph (networkx.classes.graph.Graph): networkx graph
        strategy (str): graph colouring method to find clique cover. (note is a heuristic alg)
        plot_graph (optional, bool): whether to plot graph
        node_attributes_dict (dict): Dictionary with nodes as keys and attributes as values

    Returns:
        colour_key_for_nodes (dict): A dictionary containing colours (sets) as keys and item as list of nodes
                                     that are completely connected by edges

    """
    comp_GRAPH = Openfermion_Get_Complemenary_Graph(Graph, plot_graph=False)

    greedy_colouring_output_dic = nx.greedy_color(comp_GRAPH, strategy=strategy, interchange=False)
    unique_colours = set(greedy_colouring_output_dic.values())

    colour_key_for_nodes = {}
    for colour in unique_colours:
        colour_key_for_nodes[colour] = [k for k in greedy_colouring_output_dic.keys()
                                        if greedy_colouring_output_dic[k] == colour]

    if plot_graph is True:
        import matplotlib.cm as cm
        colour_list = cm.rainbow(np.linspace(0, 1, len(colour_key_for_nodes)))
        pos = nx.circular_layout(Graph)

        for colour in colour_key_for_nodes:
            nx.draw_networkx_nodes(Graph, pos,
                                   nodelist=[node for node in colour_key_for_nodes[colour]],
                                   node_color=colour_list[colour].reshape([1,4]),
                                   node_size=500,
                                   alpha=0.8)


        # labels = {node: node for node in list(Graph.nodes)}
        seperator = ' '
        labels = {node: seperator.join([tup[1] + str(tup[0]) for tup in node[0]]) for node in list(Graph.nodes)}

        nx.draw_networkx_labels(Graph, pos, labels)  # , font_size=8)

        nx.draw_networkx_edges(Graph, pos, width=1.0, alpha=0.5)
        plt.show()

    return colour_key_for_nodes

def Convert_Clique_Cover_to_QubitOp(clique_cover_dict):
    from openfermion.ops import QubitOperator

    qubit_op_list_clique={}
    for key in clique_cover_dict:
        qubit_op_list=[]
        for PauliStr_const in clique_cover_dict[key]:
            PauliStrs, const = PauliStr_const
            Op = QubitOperator(PauliStrs, const)
            qubit_op_list.append(Op)
        qubit_op_list_clique[key] = qubit_op_list

    return qubit_op_list_clique

def Convert_Clique_Cover_to_str(clique_cover_dict):
    from openfermion.ops import QubitOperator

    qubit_op_list_clique={}
    for key in clique_cover_dict:
        qubit_op_list=[]
        for PauliStr_const in clique_cover_dict[key]:

            PauliStrs, const = PauliStr_const
            PauliStr_list = [''.join(map(str, [element for element in tupl[::-1]])) for tupl in PauliStrs]
            PauliWord = ' '.join(PauliStr_list)

            qubit_op_list.append((PauliWord, const))
        qubit_op_list_clique[key] = qubit_op_list

    return qubit_op_list_clique

class Openfermion_Hamiltonian_Graph():

    def __init__(self, QubitHamiltonian):

        self.QubitHamiltonian = QubitHamiltonian
        self.Graph = nx.Graph()

    def _Get_hashable_Hamiltonian(self):
        # networkX requires hashable object... therefore concert QubitHamiltonian to hashable form

        self.QubitHamiltonianFrozen = tuple(frozenset((PauliStr, const) for op in self.QubitHamiltonian \
                                                      for PauliStr, const in op.terms.items()))

    def _Build_Graph_nodes(self, plot_graph=False):

        self.Graph = Openfermion_Build_Graph_Nodes(self.QubitHamiltonianFrozen, self.Graph, plot_graph=plot_graph)

    def _Build_Graph_edges(self, commutativity_flag, plot_graph=False):

        self.Graph = Openfermion_Build_Graph_Edges_COMMUTING_QWC_AntiCommuting(self.Graph, self.QubitHamiltonianFrozen,
                                                                               commutativity_flag, plot_graph = plot_graph)

    def _Colour_Graph(self, Graph_colouring_strategy='largest_first', plot_graph=False):

        output_sets = Openfermion_Get_clique_cover(self.Graph, strategy=Graph_colouring_strategy, plot_graph=plot_graph)

        return output_sets

    def Get_Clique_Cover_as_QubitOp(self, commutativity_flag, Graph_colouring_strategy='largest_first', plot_graph=False):
        self.Graph.clear()
        self._Get_hashable_Hamiltonian()
        self._Build_Graph_nodes(plot_graph=plot_graph)
        self._Build_Graph_edges(commutativity_flag, plot_graph=plot_graph)
        output_sets = self._Colour_Graph(Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
        qubitOperator_list = Convert_Clique_Cover_to_QubitOp(output_sets)
        return qubitOperator_list

    def Get_Clique_Cover_as_Pauli_strings(self, commutativity_flag, Graph_colouring_strategy='largest_first', plot_graph=False):
        self.Graph.clear()
        self._Get_hashable_Hamiltonian()
        self._Build_Graph_nodes(plot_graph=plot_graph)
        self._Build_Graph_edges(commutativity_flag, plot_graph=plot_graph)
        output_sets = self._Colour_Graph(Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
        qubitOperator_list_str = Convert_Clique_Cover_to_str(output_sets)
        return qubitOperator_list_str